# Flow control

_Note_: ensure that students copy, by hand and on paper, the various definitions written by the teacher on the whiteboard. It is strongly advised to ask students *not* to use a laptop, as it will prove distracting.

## Input
- let us introduce yet another instruction
- we want to make programs interactive and capable of handling unpredictable events
- this makes them much more interesting, but also much more complex!
- `input` "somehow" reads a string from the user
- let us assume that in the state `S` we also have the sequence of all inputs the user will input (so we turn `S` into an "oracle")
    - `S` = $\{ \dots, input := \{ next := "\dots", rest := \{ next := "\dots", rest := \dots \} \} \} $
    - `S[input][next]` reads the next user input
    - `S[input := S[input][rest]` discards the next user input
    - show an example of reading the next input when there are two inputs available
    - show an example of reading the next-next input when there are two inputs available
    - why do we do this? To abstractly model how input works, we cannot write, on paper, that we wait on user input
    - we assume that we already know the inputs (they will indeed happen at some time!), and already store them in the state
    - then we can run the program and describe its exact semantics
- `eval(input(), S)` $\rightarrow$ `S[input], S[input := S[input][rest]`
- show some examples of programs with input:
    - `x := input(); y := input(); z := x + " " + y + "!"; done` where $input := \{ next := "Hello", rest := \{ next := "World", rest := \dots \} \}$
    - `x := input(); y := input(); z := int(x) + int(y); done` where $input := \{ next := "3", rest := \{ next := "5", rest := \dots \} \}$

## Conditionals
- in the presence of input, our programs become, indeed, less predictable
- we cannot know in advance the behaviour of the program before observing the input
- based on the input, it might be desirable to perform some operations instead of some others
- we now introduce the basic instruction to perform choices, the *conditional* instruction
- `if c then p else q` combines together three pieces of code, which it then coordinates together to perform a unique function
    - `c` is a boolean expression, that is an expression which will evaluate to a boolean value `True` or `False` (other values make no sense and will stop evaluation of the program and yield an error)
    - `p` is a full-blown sub-program that will be ran if `c` evaluates to `True`
    - `q` is a full-blown sub-program that will be ran if `c` evaluates to `False`
- the semantics of `if` are not particularly intricated:
    - `eval(<if True then p else q>, S)` $\rightarrow$ `<p>, S`
    - `eval(<if False then p else q>, S)` $\rightarrow$ `<q>, S`
    - `eval(<if C then p else q>, S)` $\rightarrow$ `error` (`C` is any non-boolean constant value)
    - `eval(<if c then p else q>, S)` $\rightarrow$ `<if eval_expr(<c>, S) then p else q>, S`
- let us see what happens when we use a conditional in a program
    - we will use curly brackets `{` and `}` to denote the boundaries of `p` and `q`
    - `x := int(input()); if x > 0 then { z := "the input was positive";done } else { z := "the input was not positive";done }; done`
    - `x := int(input()); if x > 0 then { z := "the input was positive";done } else { if x < 0 then { z := "the input was negative";done } else { z := the input was zero;done} }; done`
- let us improve readability of our programs: this only affects the syntax of the program, but not the semantics
    - `x := int(input()); if x > 0 then { z := "the input was positive";done } else { if x < 0 then { z := "the input was negative";done } else { z := the input was zero;done} }; done` is getting hard to read
    - the `done` instruction will remain implicit
        - `x := int(input()); if x > 0 then { z := "the input was positive" } else { if x < 0 then { z := "the input was negative" } else { z := the input was zero } }` is already a bit shorter
    - instead of `;` we will use new lines (and also for the `then` and `else` sub-programs):
        - ```x := int(input())
if x > 0 then { 
z := "the input was positive" 
} else { 
if x < 0 then { 
z := "the input was negative" 
} else { 
z := the input was zero } }``` is getting almost readable
    - instead of `{` `}` we will use indentation, that is move sub-programs to the right (with spaces/tabs) in order to specify nesting
        - we get to the much prettier
        
```
x := int(input())
if x > 0 then
  z := "the input was positive" 
else
  if x < 0 then { 
    z := "the input was negative" 
  else
    z := the input was zero
```

    - depending on what we are doing, we will use the one or the other syntax (one-liner vs formatted) without thinking about it
    - repeat after me: *changes in syntax are not relevant, it is the semantics that count*
- sometimes we need to chain multiple decisions together:
```
todo := input()
x := int(input())
y := int(input())
if todo == "add" then
  z := x + y
else
  if todo == "sub" then
    z := x - y
  else
    if todo == "mul" then
      z := x * y
    else
      if todo == "div" then
        z := x / y
      else
        z := 0
```
- fortunately, many languages nowadays offer a simple way to put an `if` as the only instruction inside an `else`: `elif`/`elseif`:
```
todo := input()
x := int(input())
y := int(input())
if todo == "add" then
  z := x + y
elif todo == "sub" then
  z := x - y
elif todo == "mul" then
  z := x * y
elif todo == "div" then
  z := x / y
else
  z := 0
```
- what is the expressive power of conditional instructions?
    - let us draw again the graph of all possible program traces
        - `p` and `q` are possible programs, so they have traces as well
        - `if c then p else q` chooses to which trace to jump        
    - a conditional instruction multiplies by two the number of possible traces of a program
    - show a simple example and its two possible traces: `x := int(input()); if x = 0 then z := "yes" else z := "no"`
- if a single conditional multiplies by two the number of traces, what if we combine two conditionals?
    - show a simple example and its four possible traces: `x := int(input()); if x % 2 = 0 then { z := "yes" } else { z := "no" }; if x % 3 = 0 then { w := "yes" } else { w := "no" }`
    - each `if` statement added after the others multiplies the number of possible traces that a program will take by two
    - $5$ conditional statements yield $2^5 = 32$ possible traces
    - $10$ conditional statements yield $2^5 = 1024$ possible traces!
- what about `elif` nesting?
    - show a simple example and its four possible traces: `x := int(input()); if x % 2 = 0 then { z := "yes" } elif x % 3 = 0 then { z := "almost" } else { z := "no" }`
    - they only add one extra possible trace
- let us produce an example with five possible traces: how would we go about it?

```
if c1 then 
  p1
else
  if c2 then
    p2
  else
    q2
  if c3 then
    p3
  else
    q3
```
    - let us count the paths to be sure 
- in conclusion, `input` gives us unpredictability, which we turn into possibilities with conditional statements: conditionals have the potential to duplicate traces, that is each conditional makes our program actually capable of behaving like two separate programs depending on what happens before it is evaluated